In [46]:
## Functions aimed to get the weather data for cities from API 
## and store it as an SQL database

# Importing libraries:
import pandas as pd
import requests
import json
from pytz import timezone
from datetime import datetime

# Main unction to retrieve data from OpenWeatherAPI and send it to SQL
def retrieve_and_send_data():
  connection_string = create_connection_string()
  cities_df = fetch_cities_data(connection_string)
  cities_weather_df = fetch_weather_data(cities_df)
  store_weather_data(cities_weather_df, connection_string)
  return "Data has been updated"

# Function to create a connection string to save data in SQL database
def create_connection_string():
  schema = "YOUR_SQL_DATABASE_NAME"
  host = "11.11.111.111"
  user = "root"
  password = "YOUR_PASSWORD"
  port = 3306
  return f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

# Function to get cities data (city name,id and lat/long) from SQL
def fetch_cities_data(connection_string):
  return pd.read_sql("cities", con=connection_string)

# Function to get weather info for cities from API
def fetch_weather_data(cities_df):
    berlin_timezone = timezone('Europe/Berlin')
    API_key='YOUR_API_KEY'
    weather_items=[]
    
    for _, city in cities_df.iterrows():
        city_id = city['city_id']
        latitude = city['latitude']
        longitude = city['longitude']
        weather_df = requests.get(f'https://api.openweathermap.org/data/2.5/forecast?lat={latitude}&lon={longitude}&appid={API_key}&units=metric')
        weather_df_json = weather_df.json()
        
        retrieval_time = datetime.now(berlin_timezone).strftime("%Y-%m-%d %H:%M:%S")
    
        for i in weather_df_json['list']:
            weather_item = {
                    "city_id": city_id,
                    "forecast_time": i.get("dt_txt",None),
                    "outlook": i["weather"][0].get('description',None),
                    "temperature": i["main"].get("temp",None),
                    "feels_like": i["main"].get('feels_like',None), 
                    "wind_speed": i["wind"].get("speed",None),
                    "rain_prob": i.get('pop',None),
                    "rain_in_last_3h":i.get('rain',{}).get('3h',0),
                    "snow_in_last_3h":i.get('snow',{}).get('3h',0),
                    "data_retrieved_at": retrieval_time
              }
            weather_items.append(weather_item)
    
    cities_weather_df = pd.DataFrame(weather_items)
    cities_weather_df["forecast_time"] = pd.to_datetime(cities_weather_df["forecast_time"])
    cities_weather_df["data_retrieved_at"] = pd.to_datetime(cities_weather_df["data_retrieved_at"])       
    
    return cities_weather_df

#Function to store weather data as an SQL database
def store_weather_data(cities_weather_df, connection_string):
  cities_weather_df.to_sql('weather',
                    if_exists='append',
                    con=connection_string,
                    index=False)

In [48]:
retrieve_and_send_data()

'Data has been updated'

In [ ]:
#libraries used (for requirements.txt on GCP)
functions-framework
SQLAlchemy
PyMySQL
pandas
requests
pytz